# Additional assignment: BERT finetuning.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%matplotlib inline
%cd /content/gdrive/My Drive/Colab Notebooks
# %mkdir DL_project
# %mkdir dataset

%cd /content/gdrive/My Drive/NLP
%cd SkolTech/hw5

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks
/content/gdrive/My Drive/NLP
/content/gdrive/My Drive/NLP/SkolTech/hw5


In [2]:
%pip install pytorch==1.3 
%pip install python==3.7
%pip install transformers==2.1.1 
%pip install tensorboard==1.9

ERROR: Could not find a version that satisfies the requirement pytorch==1.3 (from versions: 0.1.2, 1.0.2)
ERROR: No matching distribution found for pytorch==1.3
ERROR: Could not find a version that satisfies the requirement python==3.7 (from versions: none)
ERROR: No matching distribution found for python==3.7


### 1. Setup Environment.​ 

Please, use exactly this environment and don’t use any additional libraries! Otherwise, your results will not be reproducible on our server!

### 2. Load ​ "bert-base-uncased" ​ model and corresponding tokenizer. 

Please, don’t use other models, for competition all implementations of finetuning should use identical pretrained
weights! Other models are not available on reproduction server.
 -  Check that tokenizer correctly preprocesses text for the selected model(remember, the selected model is trained on lowercased texts!)

In [3]:
from transformers import BertTokenizer
from transformers import AdamW, WarmupLinearSchedule

import torch
from torch.utils.data import DataLoader, Dataset


import pandas as pd
import numpy as np 
import re
from tqdm import tqdm_notebook
import string
from scipy.sparse import csr_matrix, hstack, issparse, coo_matrix
from IPython.display import clear_output
import matplotlib.pyplot as plt
import time

In [4]:
device = torch.device('cuda')
n_gpu = torch.cuda.device_count()

for i in range(n_gpu):
    print(torch.cuda.get_device_name(i))
    
torch.manual_seed(117)

Tesla P100-PCIE-16GB


In [0]:
BERT_MODEL_NAME = "bert-base-uncased"

In [0]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME, do_lower_case=True)

In [0]:
def read_data(path):
    data_path = '../FILIMDB/'
    data_path += path
    data= []
    with open(data_path) as file:
        for line in file:
            data.append(line.strip())
    return data

In [0]:
train_texts, test_texts, dev_texts = read_data('train.texts'),read_data('test.texts'),read_data('dev.texts')
dev_b_texts, test_b_texts = read_data('dev-b.texts'),read_data('test-b.texts')
train_labels , dev_labels, dev_b_labels =  read_data('train.labels'), read_data('dev.labels'), read_data('dev-b.labels')


In [0]:
y_train = np.array([1 if label == 'pos' else 0 for label in train_labels ])
y_dev = np.array([1 if label == 'pos' else 0 for label in dev_labels ])
y_dev_b = np.array([1 if label == 'pos' else 0 for label in dev_b_labels ])

In [0]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(categories=[range(2)], sparse=False)
y_enc_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_enc_dev = encoder.fit_transform(y_dev.reshape(-1, 1))
y_enc_dev_b = encoder.fit_transform(y_dev_b.reshape(-1, 1))

In [0]:
stopwords_eng = set(["them", "she", "wasn", "wasn't", "having", "ll", "hadn", "nor", "why", "these", "she's", "both", "you're", "an", "between", "myself", "because", "being", "ve", "while", "over", "whom", "isn", "shouldn", "wouldn't", "been", "d", "themselves", "does", "most", "below", "his", "you'll", "further", "there", "was", "ain", "doesn't", "each", "couldn", "which", "that'll", "down", "won't", "than", "y", "should've", "have", "until", "their", "through", "ma", "before", "is", "yours", "so", "up", "hasn", "doesn", "him", "very", "if", "mustn", "or", "it's", "too", "re", "mustn't", "as", "now", "isn't", "mightn't", "those", "other", "above", "who", "do", "the", "wouldn", "some", "this", "for", "don", "me", "any", "what", "theirs", "weren't", "mightn", "aren", "ours", "your", "didn't", "shan", "shouldn't", "off", "has", "just", "himself", "herself", "m", "we", "by", "aren't", "yourselves", "again", "after", "you've", "you", "how", "such", "can", "o", "were", "not", "they", "out", "few", "with", "i", "be", "haven't", "are", "s", "hadn't", "had", "our", "don't", "in", "needn't", "but", "during", "weren", "it", "will", "a", "did", "of", "shan't", "ourselves", "and", "no", "doing", "yourself", "at", "then", "you'd", "he", "once", "about", "where", "more", "only", "into", "same", "my", "hers", "t", "when", "its", "own", "here", "all", "won", "needn", "under", "br", "that", "couldn't", "from", "against", "itself", "am", "should", "her", "on", "to", "didn", "haven", "hasn't","i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "you're", "you've", "you'll", "you'd", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "she's", "her", "hers", "herself", "it", "it's", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "that'll", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "don't", "should", "should've", "now", "d", "ll", "m", "o", "re", "ve", "y", "ain", "aren", "aren't", "couldn", "couldn't", "didn", "didn't", "doesn", "doesn't", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "isn", "isn't", "ma", "mightn", "mightn't", "mustn", "mustn't", "needn", "needn't", "shan", "shan't", "shouldn", "shouldn't", "wasn", "wasn't", "weren", "weren't", "won", "won't", "wouldn", "wouldn't", "br"])
def preprocess(text,stop_words=False,punctuation=False, lower=True):
    """
    Preprocess the data
    
    stop_words: flag to remove stopwords
    punctuation: flag to remove punctuation
    """
    new_text = []
    for num,value in enumerate(text):
        if lower:
            value = value.lower()
        
        for char in set(string.punctuation):
            value = value.replace(char," "+char+" ")

        if stop_words:
            pp = " ".join([word for word in value.split() ])#if word not in stopwords_eng
            if punctuation:
                punctn = string.punctuation+'…–—‘“‚„«»'
                pp = pp.translate(str.maketrans('', '',punctn ))
                new_text.append(" ".join(pp.split()))
            else:
                new_text.append(pp)
        else:
            new_text.append(" ".join(value.split())) # remove multiple spaces
             
        
    return new_text

In [0]:
train_texts = preprocess(train_texts, punctuation=True)
test_texts = preprocess(test_texts, punctuation=True)
dev_texts = preprocess(dev_texts, punctuation=True)

dev_b_texts = preprocess(dev_b_texts, punctuation=True)
test_b_texts = preprocess(test_b_texts, punctuation=True)


In [0]:
train_texts = preprocess(train_texts, stop_words=True,punctuation=True)
test_texts = preprocess(test_texts, stop_words=True,punctuation=True)
dev_texts = preprocess(dev_texts, stop_words=True,punctuation=True)

dev_b_texts = preprocess(dev_b_texts, stop_words=True,punctuation=True)
test_b_texts = preprocess(test_b_texts, stop_words=True,punctuation=True)

In [14]:
train_texts[0]

'if the myth regarding broken mirrors would be accurate everybody involved in this production would now face approximately 170 years of bad luck because there are a lot of mirrors falling to little pieces here if only the script was as shattering as the glass then the broken would have been a brilliant film now it s sadly just an overlong derivative and dull movie with only just a handful of remarkable ideas and memorable sequences sean ellis made a very stylish and elegantly photographed movie but the story is lackluster and the total absence of logic and explanation is really frustrating i got into a discussion with a friend regarding the basic concept and meaning of the film he thinks ellis found inspiration in an old legend claiming that spotting your doppelganger is a foreboding of how you re going to die interesting theory but i m not familiar with this legend and couldn t find anything on the internet about this neither personally i just think the broken is yet another umpteenth

In [15]:

tokens = tokenizer.tokenize(train_texts[0])

print(tokens)

['if', 'the', 'myth', 'regarding', 'broken', 'mirrors', 'would', 'be', 'accurate', 'everybody', 'involved', 'in', 'this', 'production', 'would', 'now', 'face', 'approximately', '170', 'years', 'of', 'bad', 'luck', 'because', 'there', 'are', 'a', 'lot', 'of', 'mirrors', 'falling', 'to', 'little', 'pieces', 'here', 'if', 'only', 'the', 'script', 'was', 'as', 'shattering', 'as', 'the', 'glass', 'then', 'the', 'broken', 'would', 'have', 'been', 'a', 'brilliant', 'film', 'now', 'it', 's', 'sadly', 'just', 'an', 'over', '##long', 'derivative', 'and', 'dull', 'movie', 'with', 'only', 'just', 'a', 'handful', 'of', 'remarkable', 'ideas', 'and', 'memorable', 'sequences', 'sean', 'ellis', 'made', 'a', 'very', 'st', '##yl', '##ish', 'and', 'elegant', '##ly', 'photographed', 'movie', 'but', 'the', 'story', 'is', 'lack', '##lus', '##ter', 'and', 'the', 'total', 'absence', 'of', 'logic', 'and', 'explanation', 'is', 'really', 'frustrating', 'i', 'got', 'into', 'a', 'discussion', 'with', 'a', 'friend',

### 3. Write ​ vectorize_ex ​ function that receives text and label for one example and required sequence length. It should return input representations for BERT (sequence of subword ids, segment ids, input mask and the label id).
 - Which special tokens were always attached to the input text during BERT pretraining? It may be a good idea to attach them during finetuning also, so BERT will see what it used to see.
 - Maximum length of an input text is limited by the number of positional embeddings in BERT. How much is it? If the text is longer, the simplest option is to truncate it (in Research part you can try other options).
 - You should pad each example to the required sequence length, because all examples in the batch should be the same length!
 - Build input mask to mask attention to the paddings - we don’t want the number of paddings to effect predictions!

## **There my BertTokenizedDataset and PadSequences for dataloaders include all functionality of vectorize_ex**

In [0]:
class BertTokenizedDataset(Dataset):
    def __init__(self, tokenizer, text_data, target_data=None, max_length=256):
        """
        Parameters
        ----------
        tokenizer : instance of BertTokenizer
        text_data : list of str
            List of input sentences
        target_data : list of int
            List of input targets
        max_length : int
            Maximum length of input sequence (length in bpe tokens)
        """
        self.tokenizer = tokenizer 
        self.max_length = max_length
        
        self.target_data = target_data
            
        self.tokens  = [] 
        self.data = []
        for line in text_data:
            line = ["[CLS]"] + self.tokenizer.tokenize(line)[:max_length - 2] + ["[SEP]"]
            self.tokens.append(line)
            self.data.append(torch.tensor(tokenizer.convert_tokens_to_ids(line)))
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        if self.target_data is not None:
            return self.data[i], self.target_data[i]
        else:
            return self.data[i]
    

In [17]:
train_dataset = BertTokenizedDataset(tokenizer, train_texts, y_train)
print('1')
dev_dataset = BertTokenizedDataset(tokenizer, dev_texts, y_dev)
print('2')
dev_b_dataset = BertTokenizedDataset(tokenizer, dev_b_texts, y_dev_b)
print('3')
test_dataset = BertTokenizedDataset(tokenizer, test_texts)
print('4')
test_b_dataset = BertTokenizedDataset(tokenizer, test_b_texts)
print('5')

1
2
3
4
5


In [18]:
train_dataset[3]

(tensor([  101,  5410,  5436,  9832,  2482, 15451, 11263, 27989,  1998, 13346,
         11865, 29256,  3494,  2012,  2034,  1045,  2245,  2023,  3185,  2001,
          2081,  2076,  1996, 26232,  2144,  1996,  3861,  3737,  2004,  2092,
          2004,  1996,  9994,  1998,  3689,  2790,  2579,  2013,  2019,  2214,
         12849,  3900,  2278,  2792,  2043,  1045,  7039,  1998,  2179,  2008,
          2009,  2001,  2428,  2081,  2004,  2397,  2004,  5989,  1045,  2001,
         22741,  2023,  2003,  2011,  2521,  2028,  1997,  1996,  5409,  1998,
          2560, 10874,  5691,  1045,  2031,  2412,  2464,  7987,  7987,  2065,
          2017,  3191,  2023,  2022,  9449,  2065,  2017,  2156,  2009,  2017,
          5949,  2051,  2043,  2017,  2071,  2031,  2589,  2242,  2062, 10990,
          2066,  3666,  6773,  4318,   102]), 0)

In [0]:
class PadSequences:
    def __init__(self, use_labels=False):
        self.use_labels = use_labels
    
    def __call__(self, batch):
        """
        Parameters
        ----------
        batch : list of objects or list of (object, label)
            Each object is list of int indexes.
            Each label is int.
        """
        data_label_batch = batch if self.use_labels else [(x, 0) for x in batch]
            
        # Sort the batch in the descending order
        max_sorted_batch = sorted(data_label_batch, key=lambda x: x[0].shape[0], reverse=True)
        sorted_batch = data_label_batch
        # Get each sequence and pad it
        sequences = [x[0] for x in sorted_batch]
        max_sorted_batch = [x[0] for x in max_sorted_batch]
        sequences_padded = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True)
        sequences_padded[:,-1] = 102
        max_lenght = len(max_sorted_batch[0])

        # Also need to store the length of each sequence
        # This is later needed in order to unpad the sequences
        lengths = torch.FloatTensor([[1] * len(x) + [0] * (max_lenght - len(x)) for x in sequences])
        # Don't forget to grab the labels of the *sorted* batch
        if self.use_labels:
            labels = torch.FloatTensor([x[1] for x in sorted_batch])
            return sequences_padded, lengths, labels
        else:
            return sequences_padded, lengths

In [0]:
BATCH_SIZE = 4

In [0]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=True))

dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=True))

dev_b_dataloader = DataLoader(dev_b_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=True))

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=False))

test_b_dataloader = DataLoader(test_b_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=False))


In [83]:
for num,x in enumerate(train_dataloader):
  if num == 2:
    break
  print(x)

(tensor([[ 101, 2065, 1996,  ..., 2000, 2019,  102],
        [ 101, 1045, 2435,  ...,    0,    0,  102],
        [ 101, 2044, 3666,  ...,    0,    0,  102],
        [ 101, 5410, 5436,  ...,    0,    0,  102]]), tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]]), tensor([0., 1., 0., 0.]))
(tensor([[  101,  2073,  1996,  ...,  4130,  1997,   102],
        [  101,  3889, 10533,  ...,     0,     0,   102],
        [  101,  1996,  3937,  ...,     0,     0,   102],
        [  101,  1045,  2031,  ...,  1996,  3185,   102]]), tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]]), tensor([1., 0., 0., 1.]))


### 4. Write main training loop. Plot learning curves: at least plot loss on train and dev (on one plot) and accuracy on train and dev (on another). For plotting Matplotlib or TensorflowX libraries will be installed on the server.
 - Use Adam optimizer. Select learning rate between 1e-6 and 1e-3.
 - Try using learning rate warmup (usually, 5-10% of the whole training time is enough) and learning rate decay. Carefully select the number of epochs (if your learning rate decays to small values too early, you will underfit).
 - What is the size of the memory in your GPU? What is the largest size you can use for BERT finetuning? Try using larger batches (32,128) via gradient accumulation trick.

In [0]:
EPOCH_AMOUNT = 2
TRAIN_LENGTH = len(train_dataset)
BATCH_SIZE = 4

LR = 5e-6

In [0]:
import torch.nn as nn
from transformers import BertModel
import torch.optim as optim
bert = BertModel.from_pretrained('bert-base-uncased')

class Bert_Sig(nn.Module):
    def __init__(self, dropout=0.1):
        super(Bert_Sig, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input_ids,input_mask):
        _, output = self.bert(input_ids,attention_mask=input_mask)
        proba = self.sigmoid(self.linear(output))
        return proba

In [0]:
bert_model = Bert_Sig()
bert_model.to('cuda')


optimizer = optim.Adam(bert_model.parameters(), lr=LR)

In [0]:
from sklearn.metrics import accuracy_score
criterion = nn.BCEWithLogitsLoss()

In [26]:
train_loss_set = []
train_acc_set = []

# trange is a tqdm wrapper around the normal python range
for _ in tqdm_notebook(range(EPOCH_AMOUNT)):


    # Training

    # Set our model to training mode (as opposed to evaluation mode)
    bert_model.train(True) 

    # Tracking variables
    tr_loss = 0
    tr_acc = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for step, batch in tqdm_notebook(enumerate(train_dataloader)):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # print(b_input_ids, b_input_mask, b_labels)
        # Clear out the gradients (by default they accumulate)
        
        # Forward pass

        outputs = bert_model(b_input_ids,b_input_mask)
        loss = criterion(outputs.squeeze(1), b_labels)
        bert_model.zero_grad()
        # loss.backward() 

        train_loss_set.append(loss.item())
        logits = outputs.squeeze(1).detach().cpu().numpy().round()
        label_ids = b_labels.detach().cpu().numpy()

        tr_acc=accuracy_score(logits,label_ids)
        train_acc_set.append(tr_acc)

        # print(f'LOSS {loss} outputs: {outputs}\n  logits:{logits} label_ids:{label_ids} tr_acc {tr_acc}')

        # Backward pass
        loss.backward()

        # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
        # torch.nn.utils.clip_grad_norm_(bert_model.parameters(), max_grad_norm)

        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

        if nb_tr_steps % 500 == 0 :
            print('steps = ',nb_tr_steps,'loss =',tr_loss/nb_tr_steps)
            print("Train acc intermediate: {}".format(np.mean(train_acc_set)))
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    print("Train acc: {}".format(np.mean(train_acc_set)))


    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    bert_model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in tqdm_notebook(dev_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
          # Forward pass, calculate logit predictions
            outputs = bert_model(b_input_ids,b_input_mask)
        # Move logits and labels to CPU
        logits = outputs.squeeze(1).detach().cpu().numpy().round()
        label_ids = b_labels.detach().cpu().numpy()

        tmp_eval_accuracy=accuracy_score(logits,label_ids)

        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    

steps =  500 loss = 0.6314704939723015
Train acc intermediate: 0.7375
steps =  1000 loss = 0.5986952036619186
Train acc intermediate: 0.80175
steps =  1500 loss = 0.5879179012378056
Train acc intermediate: 0.8225
steps =  2000 loss = 0.5812911474108696
Train acc intermediate: 0.8345
steps =  2500 loss = 0.5766620802640915
Train acc intermediate: 0.844
steps =  3000 loss = 0.5735975114007791
Train acc intermediate: 0.84975
steps =  3500 loss = 0.5719046158364841
Train acc intermediate: 0.8517142857142858
Train loss: 0.5718619774977366
Train acc: 0.8519333333333333


Validation Accuracy: 0.8916


steps =  500 loss = 0.5494365622401237
Train acc intermediate: 0.8574705882352941
steps =  1000 loss = 0.5470297164916992
Train acc intermediate: 0.8624736842105263
steps =  1500 loss = 0.548076887011528
Train acc intermediate: 0.8659523809523809
steps =  2000 loss = 0.5465896734297275
Train acc intermediate: 0.8698260869565217
steps =  2500 loss = 0.5463857264637947
Train acc intermediate: 0.87312
steps =  3000 loss = 0.546352860858043
Train acc intermediate: 0.8751481481481481
steps =  3500 loss = 0.5467173394390515
Train acc intermediate: 0.8765862068965518
Train loss: 0.5469973384380341
Train acc: 0.8772666666666666


Validation Accuracy: 0.9008



In [0]:
def train(bert_model, train_dataloader, optimizer, scheduler, EPOCH_AMOUNT, with_clipping=False,max_grad_norm_clipping=1.0):
    train_loss_set = []
    train_acc_set = []
    # trange is a tqdm wrapper around the normal python range
    for _ in tqdm_notebook(range(EPOCH_AMOUNT)):
        # Training

        # Set our model to training mode (as opposed to evaluation mode)
        bert_model.train() 

        # Tracking variables
        tr_loss = 0
        tr_acc = 0
        nb_tr_examples, nb_tr_steps = 0, 0

        # Train the data for one epoch
        for step, batch in tqdm_notebook(enumerate(train_dataloader)):
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            
            # Forward pass
            outputs = bert_model(b_input_ids,b_input_mask)
            loss = criterion(outputs.squeeze(1), b_labels)
            bert_model.zero_grad()

            train_loss_set.append(loss.item())    
            logits = outputs.squeeze(1).detach().cpu().numpy().round()
            label_ids = b_labels.detach().cpu().numpy()

            tr_acc=accuracy_score(logits,label_ids)
            train_acc_set.append(tr_acc)
            # Backward pass
            loss.backward()

            if with_clipping:
                # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
                torch.nn.utils.clip_grad_norm_(bert_model.parameters(), max_grad_norm_clipping)

            # Update parameters and take a step using the computed gradient
            optimizer.step()

            # Update tracking variables
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1

            if nb_tr_steps % 500 == 0 :
                print('steps = ',nb_tr_steps,'loss =',tr_loss/nb_tr_steps)
                print("Train acc intermediate: {}".format(np.mean(train_acc_set)))

        print("Train loss: {}".format(tr_loss/nb_tr_steps))

    plt.figure(figsize=(15,8))
    plt.subplot(121)
    plt.title("Training loss")
    plt.xlabel("Batch")
    plt.ylabel("Loss")
    plt.plot(train_loss_set)
    plt.subplot(122)
    plt.title("Training accuracy")
    plt.xlabel("Batch")
    plt.ylabel("Accuracy")
    plt.plot(train_acc_set)
    plt.show()


In [0]:
def test(bert_model, dataloader, name_dtlder='Test'):
    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    bert_model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in tqdm_notebook(dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
          # Forward pass, calculate logit predictions
          outputs = bert_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy().round()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1


    print("{} Accuracy: {}".format(name_dtlder,eval_accuracy/nb_eval_steps))

In [0]:
def predict(bert_model, dataloader):
      # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    bert_model.eval()

    all_outputs = np.array([])
    # Evaluate data for one epoch
    for batch in tqdm_notebook(dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        if len(batch) == 2:
            b_input_ids, b_input_mask = batch
        elif len(batch) == 3:
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        outputs = bert_model(b_input_ids,b_input_mask)
        # Move logits and labels to CPU
        logits = outputs.squeeze(1).detach().cpu().numpy().round()
        all_outputs = np.append(all_outputs,logits)
    return all_outputs

In [86]:
preds_train = predict(bert_model,train_dataloader)
preds_dev = predict(bert_model,dev_dataloader)
preds_dev_b = predict(bert_model,dev_b_dataloader)

In [87]:
accuracy_score(y_train,preds_train)

0.9298666666666666

In [88]:
accuracy_score(y_dev,preds_dev)

0.9008

In [89]:
accuracy_score(y_dev_b,preds_dev_b)

0.832